In [ ]:
#This is the notebook for tools_project

In [1]:
# scraping libraries to read html code
import requests
from bs4 import BeautifulSoup
import json

In [2]:
# pandas library to create dataframe
import pandas as pd
import numpy as np

In [69]:
# Direct to initial results
url = 'https://www.yelp.com/search?find_desc='
keyword = input('What do you want to eat ? ')
address = input('Where are you ? ')

url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
url

What do you want to eat ? ramen
Where are you ? new york


'https://www.yelp.com/search?find_desc=ramen&find_loc=new york'

In [70]:
# Checking if the url is "valid", the status code should be 200
response = requests.get(url)
response.status_code

200

In [71]:
# extract the html code from the web page = https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York,+NY&start=30
results_page = BeautifulSoup(response.content,'lxml')

In [72]:
# creating a list of businesses with the html code of the business as elements of the list
businesses = results_page.find_all('a',class_ = "biz-name js-analytics-click")

In [73]:
businesses

[<a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="_ZA2zJZ51LoxIKbwDKQEJw" href="/adredir?ad_business_id=OLY3Ppf7PHFeT0j_O6ptVg&amp;campaign_id=-PWZfTF3CdSDnl6rI5g44A&amp;click_origin=search_results&amp;placement=above_search&amp;redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fzen-6-new-york&amp;request_id=f23301c110c0f051&amp;signature=dc80f1987b57ac450d2589d4cd303838bcc0e948a2cd6381ac21403c516a5271&amp;slot=0"><span>Zen 6</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="xJJUNXIR1JECvREs7j-Mzg" href="/biz/ramen-hood-new-york?osq=ramen"><span>Ramen Hood</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="dzIqS1W6xuk3BBO60GiRMg" href="/biz/ippudo-ny-new-york-7?osq=ramen"><span>Ippudo NY</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="WXGLNKrunAEuV0QVBYwiKQ" href="/biz/totto-ramen-new-york-1

In [74]:
# selecting the first element (a particular restaurant in our case) of the list businesses and looking in the
#ignore the first element of the businesses list, it is an ad by yelp, regular list start from the second (index = 1)
# html code for its name
business = businesses[1]
name = business.find('span').get_text()

# getting the link to the yelp page this particular restaurant
link = 'https://www.yelp.com' + business.get('href')

In [75]:
print(name,link)

Ramen Hood https://www.yelp.com/biz/ramen-hood-new-york?osq=ramen


In [76]:
# extracting the html code of the web page associated with the first business of our list businesses
response_business = requests.get(link)
results_business_page = BeautifulSoup(response_business.content,'lxml')

In [77]:
# looking in the html page of the restaurant for latitude and longitude coordinates of the restaurant
# the json object json_map contains these coordinates
json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')

# we can convert this json object to a python dictionnary using the following method
dict_map = json.loads(json_map)

# the relevant keys in this dict are "markers", "location" and "latitude" and "longitude"
latitude = dict_map["markers"][1]["location"]["latitude"]
longitude = dict_map["markers"][1]["location"]["longitude"]

In [78]:
json_map

'{"serviceAreas": [], "moMapPossible": true, "scrollwheelZoom": false, "zoomControlPosition": "top_right", "minZoomlevel": null, "isFullBleed": false, "maxZoomlevel": null, "zoom": 15, "library": "google", "fitToGeobox": false, "hoods": [], "adPinColor": null, "markers": [{"location": null, "key": "directions_marker", "icon": {"name": "directions", "anchorOffset": [12, 32], "activeOrigin": [0, 0], "scaledSize": [24, 32], "regularUri": "https://s3-media4.fl.yelpcdn.com/assets/srv0/yelp_maps/79f63ebc20db/assets/img/directions@2x.png", "size": [24, 32], "activeUri": "https://s3-media4.fl.yelpcdn.com/assets/srv0/yelp_maps/7249ab345ac8/assets/img/directions_highlighted@2x.png", "regularOrigin": [0, 0]}}, {"resourceType": "business", "url": "/biz/ramen-hood-new-york", "resourceId": "oy_FqW6XO8S06pg6nHbhog", "shouldOpenInNewTab": false, "location": {"latitude": 40.7223112701284, "longitude": -73.9973774072879}, "key": "starred_business", "hovercardId": "xJJUNXIR1JECvREs7j-Mzg", "icon": {"name

In [79]:
print(latitude,longitude)

40.7223112701284 -73.9973774072879


In [80]:
# The category can be found in the 'a' tag under "span", class_ = 'category-str-list' tag, 
category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()

In [81]:
category

'Ramen'

In [82]:
# the price range ('$','$$','$$$', or '$$$$') can be found in the "span", class_ = 'business-attribute price-range' tag
# the price range is not always available, that is why we need to catch errors 
# if necessary (hence the try: except) structure

try:
    price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
except:
    price_range = None

In [83]:
print(price_range)

$$


In [84]:
# the ratings associated to a particular restaurant are in the "div", itemprop = 'aggregateRating' tag
# the rating value (between 0 and 5) can be accessed through ratings (= results_business_page.find("div", 
# itemprop = 'aggregateRating') at the tag "meta" and the attribute 'content'
# the number of reviews is available through ratings at the 'span' tag

ratings = results_business_page.find("div", itemprop = 'aggregateRating')
rating_value = ratings.find("meta").get('content')
review_count = ratings.find("span").get_text()

In [85]:
print(rating_value,review_count)

4.5 31


In [86]:
# the phone number can be accessed at the tag 'span',itemprop = "telephone"
# phone numbers are not always available so we need to catch errors using the try:... except: structure

try:
    phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
except:
    phone_number = None   

In [87]:
print(phone_number)

In [88]:
# We want to know if the restaurant is currently open: this information is 
# given at the 'span', class_ = "nowrap extra open" tag. This information is not always 
# available that is why we use the try: ... except:... structure

try:
    open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
except:
    open_ = None

In [89]:
print(open_)

None


In [119]:
try:
    addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
except:
    addr = None
addr

"62 Spring StChef's Club CounterNew York, NY 10012"

In [91]:
try:
    web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
except:
    web = None
web

'chefsclubcounter.com'

In [82]:
#want to put all infomations we want to get from one business into one funtion
#input: url link
#output: all details we tested above
def get_info(link):
    response_business = requests.get(link)
    results_business_page = BeautifulSoup(response_business.content,'lxml')
    
    try:
        addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
    except:
        addr = None

#     try:
#         json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')
#         dict_map = json.loads(json_map)
#         lat = dict_map["markers"][1]["location"]["latitude"]
#         lon = dict_map["markers"][1]["location"]["longitude"]
#     except: 
#         json_map = None 

    try: 
        category = results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()
    except:
        category = None
        
    try:
        price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
    except:
        price_range = None
        
    try: 
        rating_value = results_business_page.find("div", {'itemprop' : "aggregateRating"}).find("meta").get('content')
    except: 
        rating_value = None
        
#     try:
#         review_count = results_business_page.find("div", {'itemprop' : "aggregateRating"}).find("span").get_text()
#     except: 
#         rating_count = None
    
    try:
        phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
    except:
        phone_number = None
        
    try:
        open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
    except:
        open_ = 'closed now'
        
    try:
        web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
    except:
        web = None
        
    return(addr,category,price_range,rating_value,phone_number,open_,web)
    

In [18]:
#test the function on cafe mogador
restaurant_url = "https://www.yelp.com/biz/cafe-mogador-brooklyn?osq=Restaurants"
get_info(restaurant_url)

('133 Wythe AveBrooklyn, NY 11211',
 40.719747,
 -73.959993,
 'Moroccan',
 '$$',
 '4.5',
 '1004',
 '(718) 486-9222',
 'Open now',
 'cafemogador.com/williamsburg')

In [94]:
#Get a list of url links of all businesses (or maybe the first 30)
#so that we could compile a list of details on, for example, the first 30 search-result restaurants
link_list = []

for i in range(1,len(businesses)):
    business = businesses[i]
    link = 'https://www.yelp.com' + business.get('href')
    link_list.append(link)

In [96]:
#Create a list of restaurant names
name_list=[]
for i in range(1,len(businesses)):
    business = businesses[i]
    name = business.find('span').get_text()
    name_list.append(name)

In [98]:
#Create a list of all restaurants' information
info_list=[]

for i in range(len(link_list)):
    info_list.append(get_info(link_list[i]))

In [100]:
#Create a dictionary showing the information of each restaurant
business_info={}

for i in range(len(name_list)):
    business_info[name_list[i]] = info_list[i]        

In [19]:
def get_businesses():
    business_list = []
    
    keyword = input('What do you want to eat ? ')
    address = input('Where are you ? ')
    
    #df = pd.DataFrame(columns = ['name','open or closed','latitude','longitude','price range','rating value','review count','category','phone','website'])
                                             
    url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
    response = requests.get(url)
    try:
        results_page = BeautifulSoup(response.content,'lxml')
    except:
        print("we didn't get back result page from yelp correctly")
        
    businesses = results_page.find_all('a',{'data-analytics-label':"biz-name"})
    business_list = []
    for business in businesses:
        name = business.find('span').get_text()
        link = 'https://www.yelp.com' + business.get('href')
        addr,category,price_range,rating_value,phone_number,open_,web= get_info(link)
        #df.loc[i] = [name,open_,latitude,longitude,price_range,rating_value,review_count,category,phone_number,web]
        business_list.append([name,open_,price_range,rating_value,category,phone_number,web])

    return(business_list)

In [89]:
#workspace
url = "https://www.yelp.com/search?find_desc=burger+&find_loc=new+york"
response = requests.get(url)
results_page = BeautifulSoup(response.content,'lxml')
businesses = results_page.find_all('a',{'data-analytics-label':"biz-name"})
business_list = []
for business in businesses:
    name = business.find('span').get_text()
    link = 'https://www.yelp.com' + business.get('href')
    addr,category,price_range,rating_value,phone_number,open_,web= get_info(link)
    #[name,open_,latitude,longitude,price_range,rating_value,review_count,category,phone_number,web]        
    #df.loc[i] = [name,open_,latitude,longitude,price_range,rating_value,review_count,category,phone_number,web]
    business_list.append([name,open_,price_range,rating_value,category,phone_number,web])
business_list


[['Ellis Cafe', 'closed now', None, None, None, None, None],
 ['Burger & Lobster',
  'closed now',
  '$$',
  '4.0',
  'Seafood',
  '(646) 833-7532',
  'burgerandlobster.com/en/locat…'],
 ['Diesel & Duke',
  'closed now',
  '$$',
  '4.5',
  'Burgers',
  '(201) 222-1322',
  'eatdiesel.com'],
 ['Walburgers',
  'closed now',
  '$$',
  '2.5',
  'Burgers',
  '(718) 942-4450',
  'wahlburgersrestaurant.com'],
 ['Burgerology',
  'closed now',
  '$$',
  '3.5',
  'Burgers',
  '(917) 745-0299',
  'burgerologyastoria.com'],
 ['The Burgary',
  'closed now',
  '$$',
  '4.5',
  'Burgers',
  '(212) 529-6900',
  'theburgary.com'],
 ['Black Iron Burger',
  'closed now',
  '$$',
  '4.0',
  'Burgers',
  '(347) 380-4731',
  'blackironburger.com'],
 ['Juanchi’s Burger',
  'closed now',
  '$$',
  '4.5',
  'Gastropubs',
  '(929) 295-0147',
  'juanchisburgersbk.com'],
 ['dumont burger',
  'closed now',
  '$$',
  '4.5',
  'Burgers',
  '(718) 388-0771',
  'dumontburgerny.com'],
 ['Benson’s NYC',
  'Open now',
  '

In [ ]:
businesses

In [20]:
get_businesses()

What do you want to eat ? burger
Where are you ? new york


[]